In [1]:
import os
!pwd

/home/gautham/E2EProjects/TextSummarisation/research


In [2]:
os.chdir("../")

In [3]:
!pwd

/home/gautham/E2EProjects/TextSummarisation


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir : Path
    data_path : Path
    tokenizer_name : Path    

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


class ConfigarationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 param_file_path = PARAMS_FILE_FILE):
        

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(param_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        """ 
        Returns : DataTransformationConfig
        """
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [9]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)
        target_encodings = self.tokenizer(text_target=example_batch["summary"], max_length=128, truncation=True)

        return {
            "input_ids" : input_encodings["input_ids"],
            "attention_mask" : input_encodings["attention_mask"],
            "labels" : target_encodings["input_ids"]
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [8]:
try:
    config = ConfigarationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.convert()

except Exception as exception:
    raise exception

[2024-03-18 11:57:31,112: INFO: common: yaml file : config/config.yaml loded successfully]
[2024-03-18 11:57:31,114: INFO: common: yaml file : params.yaml loded successfully]
[2024-03-18 11:57:31,116: INFO: common: Created directory at : artifacts]
[2024-03-18 11:57:31,118: INFO: common: Created directory at : artifacts/data_transformation]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 89173.25 examples/s] 
